In [24]:
import pandas as pd
npr_articles = pd.read_csv('national-public-radio.csv')
npr_articles.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
document_term_matrix = cv.fit_transform(npr_articles['Article'])
document_term_matrix

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [5]:
print(len(cv.get_feature_names()))
import random
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(cv.get_feature_names()[random_word_id])

54777
nantes
chromatic
tali
poor
irna
breakdowns
leaguers
anthropology
balsam
cadaver


In [6]:
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=7,random_state=42)
LDA.fit(document_tearm_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

<h3>Top Words Per Topic</h3>

In [8]:
len(LDA.components_)

7

In [9]:
LDA.components_

array([[8.64332806e+00, 2.38014333e+03, 1.42900522e-01, ...,
        1.43006821e-01, 1.42902042e-01, 1.42861626e-01],
       [2.76191749e+01, 5.36394437e+02, 1.42857148e-01, ...,
        1.42861973e-01, 1.42857147e-01, 1.42906875e-01],
       [7.22783888e+00, 8.24033986e+02, 1.42857148e-01, ...,
        6.14236247e+00, 2.14061364e+00, 1.42923753e-01],
       ...,
       [3.11488651e+00, 3.50409655e+02, 1.42857147e-01, ...,
        1.42859912e-01, 1.42857146e-01, 1.42866614e-01],
       [4.61486388e+01, 5.14408600e+01, 3.14281373e+00, ...,
        1.43107628e-01, 1.43902481e-01, 2.14271779e+00],
       [4.93991422e-01, 4.18841042e+02, 1.42857151e-01, ...,
        1.42857146e-01, 1.43760101e-01, 1.42866201e-01]])

In [10]:
len(LDA.components_[0])

54777

In [16]:
single_topic = LDA.components_[0]
# Returns the indices that would sort this array.
print(single_topic.argsort())
# Word least representative of this topic
print(single_topic[18302])
# Word most representative of this topic
print(single_topic[42993])
# Top 10 words for this topic:
print(single_topic.argsort()[-10:])

[ 2475 18302 35285 ... 22673 42561 42993]
0.14285714309286987
6247.245510521049
[33390 36310 21228 10425 31464  8149 36283 22673 42561 42993]


In [17]:
top_word_indices = single_topic.argsort()[-10:]
for index in top_word_indices:
    print(cv.get_feature_names()[index])

new
percent
government
company
million
care
people
health
said
says


In [40]:
topic_map={}
for index,topic in enumerate(LDA.components_):
    print(f'The Top 15 Words for Topic #{index} :')
    topic_map[index]=[cv.get_feature_names()[i] for i in topic.argsort()[-15:]]
    print(topic_map[index])
    print('\n')

The Top 15 Words for Topic #0 :
['companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']


The Top 15 Words for Topic #1 :
['military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']


The Top 15 Words for Topic #2 :
['way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']


The Top 15 Words for Topic #3 :
['time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']


The Top 15 Words for Topic #4 :
['voters', 'vote', 'election', 'party', 'new', 'obama', 'court', 'republican', 'campaign', 'people', 'state', 'president', 'clinton', 'said', 'trump']


The Top 15 Words for Topic #5 :
['years', 'going', 've', 'life', 'don', 'new', 'way', 'music', 'really', 'time', 'kno

<h3>Attaching Topic Labels to News Articles</h3>

In [25]:
document_term_matrix.shape

(11992, 54777)

In [26]:
len(npr_articles)

11992

In [35]:
topic_model_results = LDA.transform(document_term_matrix)
print("Topic Model Result Dimentions :",topic_model_results.shape)
print("Topic Model Result for 1st Article :",topic_model_results[0])
print("Rounded Topic Model Result for 1st Article :",topic_model_results[0].round(2))
topic_model_results[0].argmax()

Topic Model Result Dimentions : (11992, 7)
Topic Model Result for 1st Article : [1.61040465e-02 6.83341493e-01 2.25376318e-04 2.25369288e-04
 2.99652737e-01 2.25479379e-04 2.25497980e-04]
Rounded Topic Model Result for 1st Article : [0.02 0.68 0.   0.   0.3  0.   0.  ]


1

<p>This means that our model thinks that the first article belongs to topic #1.</p>

<h3>Combining with Original Data</h3>

In [37]:
topic_model_results.argmax(axis=1)

array([1, 1, 1, ..., 3, 4, 0])

In [43]:
npr_articles['Topic'] = topic_model_results.argmax(axis=1)
npr_articles['Words For Topic'] = npr_articles.apply(lambda row: topic_map[row.Topic], axis = 1)
npr_articles.head(10)

,Article,Topic,Words For Topic
0,"In the Washington of 2016, even when the polic...",1,"[military, house, security, russia, government..."
1,Donald Trump has used Twitter — his prefe...,1,"[military, house, security, russia, government..."
2,Donald Trump is unabashedly praising Russian...,1,"[military, house, security, russia, government..."
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1,"[military, house, security, russia, government..."
4,"From photography, illustration and video, to d...",2,"[way, world, family, home, day, time, water, c..."
5,I did not want to join yoga class. I hated tho...,3,"[time, new, don, years, medical, disease, pati..."
6,With a who has publicly supported the debunk...,3,"[time, new, don, years, medical, disease, pati..."
7,"I was standing by the airport exit, debating w...",2,"[way, world, family, home, day, time, water, c..."
8,"If movies were trying to be more realistic, pe...",3,"[time, new, don, years, medical, disease, pati..."
9,"Eighteen years ago, on New Year’s Eve, David F...",2,"[way, world, family, home, day, time, water, c..."
